# Segmenting and Clustering Neighbourhoods in Toronto

### importing required libraries and tools

In [2]:
!pip install beautifulsoup4
!pip install lxml

import pandas as pd 
import numpy as np 
import random 
import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 

from IPython.display import display_html
    
!conda install -c conda-forge folium=0.5.0 --yes
import folium

from bs4 import BeautifulSoup

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\a\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0




geopy-2.1.0          | 64 KB     |            |   0% 
geopy-2.1.0          | 64 KB     | ##5        |  25% 
geopy-2.1.0          | 64 KB     | ########## | 100% 

geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 3

## Scraping the Wikipedia page for the table of postal codes of Canada

In [45]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [46]:
soup = BeautifulSoup(source, 'lxml')

In [47]:
table_contents = []
table = soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [48]:
df = pd.DataFrame(table_contents)

In [49]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [90]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [51]:
print('The number of rows in the dataframe:', df.shape[0])

The number of rows in the dataframe: 103


## Importing the csv file with latitudes and longitudes for neighbourhoods in Canada

In [52]:
lat_lon = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [53]:
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two tables for getting the Latitudes and Longitudes for neighbourhoods in Canada

In [54]:
lat_lon.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

In [57]:
common_df = pd.merge(df,lat_lon, on = 'PostalCode')
common_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


##  Only boroughs that contain the word Toronto will be processed

In [61]:
df_2 = common_df[common_df['Borough'].str.contains('Toronto', regex = False)]
df_2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


### Visualizing the Map of Toronto with Neighbourhoods from above data_frame

### getting the Toronto coordinates

In [62]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### creating the map

In [66]:
toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat,lng,borough,neighbourhood in zip(df_2['Latitude'],df_2['Longitude'],df_2['Borough'],df_2['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'black',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.7,
    parse_html = False).add_to(toronto)
    
toronto

## Clustering the neighbourhoods using K-Means method

In [84]:
k = 5

toronto_clusters = df_2.drop(['PostalCode','Borough','Neighborhood'],1)

kmeans = KMeans(n_clusters = k, random_state = 0).fit(toronto_clusters)

kmeans.labels_
df_2.insert(0, 'Cluster_Labels', kmeans.labels_)

In [85]:
kmeans.labels_

array([0, 0, 0, 3, 0, 0, 2, 0, 4, 3, 0, 2, 3, 0, 2, 3, 0, 3, 1, 1, 1, 1,
       4, 1, 2, 4, 1, 2, 4, 1, 2, 1, 0, 0, 0, 0, 0, 0, 3])

In [87]:
df_2.head(10)

,Cluster_Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,2,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,3,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


### creating the map

In [88]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

x = np.arange(k)
ys = [i + x + (i * x)** 2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood'], df_2['Cluster_Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters